# Single mode fibre
This uses the theoretical result of the single mode optical fibre in order to form a point of comparison to the Fenics solver. 

In [2]:
from __future__ import division
import numpy as np
import matplotlib.pylab as plt
from scipy.constants import c,pi
from scipy.special import jv,kv
from scipy.optimize import fsolve
from math import atan
from scipy.integrate import dblquad 

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
#Inputs:
n1 = 1.444
n0 = 1.445
lamda = 1.55e-6
a = 1e-5
V  = a*2*pi/lamda*(n0**2 - n1**2)**0.5
Delta = (n0**2 - n1**2)/(2*n1**2)

In [4]:
def system(vec,V,Delta):
    u,w = vec
    return u**2+w**2 - V**2, jv(0,u)/(u*jv(1,u)) - (1-Delta)*kv(0,w)/(w*kv(1,w))

In [5]:
u,w = fsolve(system,[1,1],args=(V,Delta))

In [6]:
neff = (((n1/u)**2 + (n0/w)**2)/((1/u)**2 + (1/w)**2))**0.5
beta = neff*2*pi/lamda

In [7]:
neff

1.4445293532400976

In [8]:
def sigma(u,w,V,Delta):
    return -1 - u**2 * w**2 * Delta * jv(0,u)/(V**2 * u * jv(1,u))


In [9]:
def cart_to_cyl(x,y,z):
    """
    Transforms cartesian coordinates to cylyndrical coordinates
    input x, y, z
    returns tuple:  r, theta, z 
    """
    try:
        return (x**2 + y**2)**0.5, atan(y/x),z
    except ZeroDivisionError:
        
        return (x**2 + y**2)**0.5, atan(np.inf),z

In [10]:
x = np.linspace(-2*a,2*a,512)
y = np.linspace(-2*a,2*a,512)

In [11]:
def electric_field(x,y,u,w,beta,a,V,psi,n):
    n+=1
    r,theta,z = cart_to_cyl(x,y,0)
    s = sigma(u,w,V,Delta)
    if r <=a:
        er = (-1j* beta* a/u)*np.cos(n*theta +psi)*(0.5*(1-s)*jv(n-1,u*r/a) - 0.5*(1+s)*jv(n+1,u*r/a))
        etheta = (1j*beta*a/u)*np.sin(n*theta+psi)*(0.5*(1-s)*jv(n-1,u*r/a)+0.5*(1+s)*jv(n+1,u*r/a))
        ez = jv(n,u*r/a)*np.cos(n*theta+psi)
    else:
        er = -1j*beta *a *jv(n,u)/(w*kv(n,w))*(0.5*(1-s)*kv(n-1,w*r/a)+0.5*(1+s)*kv(n+1,w*r/a))*np.cos(n*theta + psi)
        etheta = 1j*beta *a *jv(n,u)/(w*kv(n,w))*(0.5*(1-s)*kv(n-1,w*r/a)-0.5*(1+s)*kv(n+1,w*r/a))*np.sin(n*theta + psi)
        ez = (jv(n,u)/w*kv(n,w))*np.cos(n*theta+psi) 
    ex = er*np.cos(theta) - etheta*np.sin(theta)
    ey =er*np.sin(theta) + etheta*np.cos(theta)
    return ex,ey,ez    

In [12]:
def plot_electric(x,y,u,w,beta,a,V):
    ele = np.zeros([3,len(x),len(y)],dtype=np.complex128)
    for i,xx in enumerate(x):
        for j, yy in enumerate(y):
            ele[:,i,j] = electric_field(xx,yy,u,w,beta,a,V,0,0)
    abss = (np.abs(ele[0,:,:])**2 + np.abs(ele[1,:,:])**2 + np.abs(ele[2,:,:])**2)**0.5
    X,Y = np.meshgrid(x,y)
    fig = plt.figure()
    plt.contourf(X,Y,abss)
    plt.show
    return 0
    

In [13]:
plot_electric(x,y,u,w,beta,a,V)

0

In [14]:
def I(y,x,u,w,beta,a,V,psi,n,po):
    ex,ey,ez = electric_field(x,y,u,w,beta,a,V,psi,n)
    return np.abs(np.abs(ex)**2 +np.abs(ex)**2 + np.abs(ex)**2)**po

In [15]:
def effective_area(a,u,w,beta,V,n):
    top = dblquad(I,-2*a,2*a,lambda x : -2*a,lambda x: 2*a,args = (u,w,beta,a,V,0,n,1))
    bottom = dblquad(I,-2*a,2*a,lambda x : -2*a,lambda x: 2*a,args = (u,w,beta,a,V,0,n,2))
    A_eff = top[0]**2/bottom[0]
    return A_eff

In [16]:
effective_area(a,u,w,beta,V,0)

/usr/local/lib/python2.7/dist-packages/scipy/integrate/quadpack.py:357: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


4.115536353963241e-10